In [7]:
import pandas as pd
import numpy as np
import sqlite3

#### 处理靶点疾病关联信息

In [8]:
target_info = tuple(set(pd.read_csv(r"D:\Herb数据库\version2\node\Target_info_in_physical_network_and_experimental.csv",index_col=0)['Gene_name']))
positive_disease_info = tuple(set(pd.read_csv(r"D:\Herb数据库\version2\Positive_Disease_Compound_pair_in_HERB.csv",index_col=0)['DisGENet_id'])|
                             set(pd.read_csv(r"D:\Herb数据库\version2\Positive_Disease_Herb_pair_in_HERB.csv",index_col=0)['DisGENet_id']))

In [3]:
disgenet = sqlite3.connect("disgenet_2020.db")
d = disgenet.cursor()
d.execute("select name from sqlite_master where type='table'")
print(d.fetchall())

[('diseaseAttributes',), ('diseaseClass',), ('disease2class',), ('geneAttributes',), ('geneDiseaseNetwork',), ('variantAttributes',), ('variantGene',), ('variantDiseaseNetwork',)]


In [33]:
d.execute("PRAGMA table_info(disease2class)")
d.fetchall()

[(0, 'diseaseNID', 'smallint(5)', 1, None, 1),
 (1, 'diseaseClassNID', 'smallint(5)', 1, None, 2)]

In [37]:
d.execute("select * from disease2class")
len(set(pd.DataFrame(d.fetchall())[0]))

18695

In [25]:
d.execute(
    '''select gd.diseaseID,gd.diseaseName,gd.type,gd.diseaseClass,gd.diseaseClassName,ga.geneID,ga.geneName,gn.source,gn.associationType,gn.score
        from (select da.diseaseNID,da.diseaseId,da.diseaseName,da.type,dc.diseaseClass,dc.diseaseClassName from diseaseAttributes da
                left join disease2class d2c
                    on da.diseaseNID=d2c.diseaseNID
                left join diseaseClass dc
                    on d2c.diseaseClassNID=dc.diseaseClassNID        
                        ) gd 
                    
                    left join geneDiseaseNetwork gn
                        on gn.diseaseNID=gd.diseaseNID
                    left join geneAttributes ga 
                        on gn.geneNID=ga.geneNID
                            where source in 
                ('UNIPROT','CTD_human','ORPHANET','CLINGEN',
                'GENOMICS_ENGLAND','CGI','PSYGENET')
                    and associationType != 'Therapeutic'
                    and ga.geneName in {};'''.format(target_info)
)
protein_disease = pd.DataFrame(d.fetchall())
protein_disease.columns = ['DisGENet_id','Disease_name','Disease_type','Disease_class','Disease_class_name','Gene_id','Gene_name','source','associationType','weight']

In [40]:
disease_info = protein_disease.iloc[:,:5].drop_duplicates()
protein_disease_edge = protein_disease[['DisGENet_id','Disease_name','Disease_type','Gene_id','Gene_name','source','associationType','weight']].drop_duplicates()

In [41]:
len(protein_disease_edge['Gene_id'].value_counts()),len(protein_disease_edge['DisGENet_id'].value_counts())

(8670, 10761)

In [42]:
disease_info.to_csv(r'D:\Herb数据库\version2\node\Disease_info_in_DisGENet.csv')
protein_disease_edge.to_csv('Protein_Disease_filtered_in_DisGENet.csv')

In [18]:
protein_disease_edge = pd.read_csv('Protein_Disease_filtered_in_DisGENet.csv',index_col=0)
common_disease_id = list(pd.read_csv('常见疾病信息（有参考文献）0820.csv',index_col=0)['DisGENet_disease_id'])
nc_disease_id = list(pd.read_excel('Protein_Disease_in_NC.xlsx')['node_1'].drop_duplicates())
useful_disease_id = list(set(common_disease_id)|set(nc_disease_id))
protein_disease_edge_filtered = protein_disease_edge.query("DisGENet_id in @useful_disease_id")

In [19]:
len(set(protein_disease_edge_filtered.DisGENet_id)),len(set(protein_disease_edge_filtered.Gene_name))

(903, 6763)

In [20]:
protein_disease_edge_filtered.to_csv('Protein_Disease_filtered_in_DisGENet_and_HERB_and_NC.csv')

In [13]:
protein_disease_edge

,DisGENet_id,Disease_name,Disease_type,Gene_id,Gene_name,source,associationType,weight
0,C0001618,Tumors of Adrenal Cortex,group,2771,GNAI2,CGI,CausalMutation,0.30
2,C0001618,Tumors of Adrenal Cortex,group,5573,PRKAR1A,CGI,CausalMutation,0.30
4,C0001618,Tumors of Adrenal Cortex,group,7157,TP53,CGI,CausalMutation,0.50
6,C0001815,Primary Myelofibrosis,disease,3717,JAK2,CGI,CausalMutation,1.00
7,C0001815,Primary Myelofibrosis,disease,4352,MPL,CGI,CausalMutation,1.00
...,...,...,...,...,...,...,...,...
276482,C4310794,"MENTAL RETARDATION, AUTOSOMAL RECESSIVE 53",disease,54872,PIGG,UNIPROT,GeneticVariation,0.70
276483,C4721610,"Carcinoma, Ovarian Epithelial",disease,6794,STK11,UNIPROT,GeneticVariation,0.36
276486,C4721610,"Carcinoma, Ovarian Epithelial",disease,999,CDH1,UNIPROT,GeneticVariation,0.40
276489,C4721806,"Carcinoma, Basal Cell",disease,5921,RASA1,UNIPROT,GeneticVariation,0.61
